# Runner Reidentification

After combining the Fronts and Backs cameras runner now we need to combine the 2 result in order to have only one rapresentation for each runner

So the steps are:

- Select the 2 runner features: the front and the back ones
- Aggregate mean of starting frame between 2 cameras
- compute similarity matrix between each runner temporally near
- assignment

## Dependencies

we firs import all the dependencies needed:

- pandas - to manipulate dataframe
- numpy - for matrix operations
- select_bib_number - custom function that select ( given feature of all relevation for one track id ) the best bib number of the runner
- aggregate_features - aggregate ( given feature of all relevation for one track id ) all the feature of a runner
- temporal_overlap - checks if
- compute_similarity
- build_similarity_matrix

In [2]:
import pandas as pd
import numpy as np

from utils import aggregate_features, build_similarity_matrix

## STEP 1 - Select and concat cameras data

In [3]:

df_FD = pd.read_csv('set_videos_2/results_static_similarity_metric/FD_thresholded_detections_aggregated_features.csv')
df_FI = pd.read_csv('set_videos_2/results_static_similarity_metric/FI_thresholded_detections_aggregated_features.csv')
df_FD_FI = pd.concat([df_FD,df_FI])

df_TD = pd.read_csv('set_videos_2/results_static_similarity_metric/TD_thresholded_detections_aggregated_features.csv')
df_TI = pd.read_csv('set_videos_2/results_static_similarity_metric/TI_thresholded_detections_aggregated_features.csv')
df_TD_TI = pd.concat([df_TD,df_TI])

Adesso abbiamo una serie di rilevazioni per le telecamere frontali, le trait feature e i numeri sono fissi per le due camere, ma abbiamo bisogno di combinare il primo e l'ultimo frame in cui compaiono, per poter fare confronti con le rilevazioni delle camere posteriori, faccio quindi media tra primo frame in cui appare il runner nella camera1 e il primo frame in cui appare nell'altra camera, faccio lo stesso per l'ultimo frame. Dato che il csv contiene informazioni aggiuntive, tolgo colonne che non servono al momento. Salvo i dati di tutti gli n corridori in una lista

In [4]:
def create_track_list(grouped_df, position):
    #get the cameras name
    if position=='frontal':
        cams=['FD','FI']
    elif position == 'rears':
        cams=['TD','TI']
    else: assert False, 'invalid camera'

    track_list = []
    for mid, group in grouped_df:
        group = group.drop(columns=['x1r','x2r','y1r','y2r','x1b','x2b','y1b','y2b', 'filename','trackid'])
        #compute the mean on start and end frame of the two cams
        start_frame_cam1 = group[group['cam'] == cams[0]]['nframe'].min()
        end_frame_cam1 = group[group['cam'] == cams[0]]['nframe'].max()
        start_frame_cam2 = group[group['cam'] == cams[1]]['nframe'].min()
        end_frame_cam2 = group[group['cam'] == cams[1]]['nframe'].max()

        start_frame_mean = (start_frame_cam1 + start_frame_cam2) // 2
        end_frame_mean = (end_frame_cam1 + end_frame_cam2) // 2

        group = group.drop(columns=['cam', 'nframe'])
        group['start_frame'] = start_frame_mean
        group['end_frame'] = end_frame_mean

        track_list.append((group.iloc[0]))#take the first in the group ( all the same )

    return track_list

In [5]:
grouped_mid_frontal = df_FD_FI.groupby("match_id")
grouped_mid_rears = df_TD_TI.groupby("match_id")
track_list_rears = create_track_list(grouped_mid_rears, position='rears')
track_list_frontals = create_track_list(grouped_mid_frontal, position='frontal')

## Step 3 - Compute similarity matrix

In [6]:
sim_matrix = build_similarity_matrix(track_list_frontals, track_list_rears, threshold=120, offset=25*5, dynamic_metric=False)

## Step 4 - Assignment

per distinguerlo dal precedene match id li genero come multipli di 1'

In [7]:
from scipy.optimize import linear_sum_assignment

runner_Frontal, runner_Reard = linear_sum_assignment(sim_matrix) # Hungarian algorithm implementation
for track in track_list_rears:
    track['final_match_id'] = None

for track in track_list_frontals:
    track['final_match_id'] = None

final_match_id = 0
for c, cons in zip(runner_Reard, runner_Frontal):
    final_match_id +=10
    track_list_rears[c]['final_match_id'] = final_match_id
    track_list_frontals[cons]['final_match_id'] = final_match_id

## Aggregate features by match id
## SAVE FEATURE ONLY


In [8]:
# convert list series into Pandas Dataframe
df_tracks_Back = pd.DataFrame(track_list_rears)
df_tracks_Back['cam'] = 'back'
df_tracks_Frontal = pd.DataFrame(track_list_frontals)
df_tracks_Frontal['cam'] = 'frontal'
#concat
df_all = pd.concat([df_tracks_Back, df_tracks_Frontal], ignore_index=True, sort=False)
grouped = df_all.groupby("final_match_id")  #group by final match id

#Feature aggregation
match_id_list = []
for match_id, group in grouped:
    feature_vector = aggregate_features(group, with_frame=False, final=True)
    match_id_list.append(feature_vector)

#now we have two list containing for each runner(trackid) his aggregated features

In [9]:
runners_FD_FI_TD_TI_features = pd.DataFrame(match_id_list)
runners_FD_FI_TD_TI_features.to_csv("set_videos_2/results_static_similarity_metric/FD_FI_TD_TI_only_features.csv", index=False)

SAVE WITH AGG FEATUREA

In [10]:
df_TD = df_TD.merge(df_tracks_Back[['match_id','final_match_id']], on='match_id', how='left')
df_TI = df_TI.merge(df_tracks_Back[['match_id','final_match_id']], on='match_id', how='left')

df_FD = df_FD.merge(df_tracks_Frontal[['match_id','final_match_id']], on='match_id', how='left')
df_FI = df_FI.merge(df_tracks_Frontal[['match_id','final_match_id']], on='match_id', how='left')


In [11]:
df_FD_no_trait = df_FD[["final_match_id", "cam","filename","nframe","trackid","x1r","y1r","x2r","y2r","x1b","y1b","x2b","y2b"]]
df_FD_agg = df_FD_no_trait.merge(runners_FD_FI_TD_TI_features, on='final_match_id', how='left')

df_FI_no_trait = df_FI[["final_match_id", "cam","filename","nframe","trackid","x1r","y1r","x2r","y2r","x1b","y1b","x2b","y2b"]]
df_FI_agg = df_FI_no_trait.merge(runners_FD_FI_TD_TI_features, on='final_match_id', how='left')

df_TD_no_trait = df_TD[["final_match_id", "cam","filename","nframe","trackid","x1r","y1r","x2r","y2r","x1b","y1b","x2b","y2b"]]
df_TD_agg = df_TD_no_trait.merge(runners_FD_FI_TD_TI_features, on='final_match_id', how='left')

df_TI_no_trait = df_TI[["final_match_id", "cam","filename","nframe","trackid","x1r","y1r","x2r","y2r","x1b","y1b","x2b","y2b"]]
df_TI_agg = df_TI_no_trait.merge(runners_FD_FI_TD_TI_features, on='final_match_id', how='left')



In [13]:
df_FD_agg.to_csv('set_videos_2/results_static_similarity_metric/FD_thresholded_detections_aggregated_features.csv', index=False)
df_FI_agg.to_csv('set_videos_2/results_static_similarity_metric/FI_thresholded_detections_aggregated_features.csv', index=False)

df_TD_agg.to_csv('set_videos_2/results_static_similarity_metric/TD_thresholded_detections_aggregated_features.csv', index=False)
df_TI_agg.to_csv('set_videos_2/results_static_similarity_metric/TI_thresholded_detections_aggregated_features.csv', index=False)